## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import random
import os
import numpy as np

# Clustering
from sklearn.cluster import KMeans

# Visualization
import matplotlib.pyplot  as plt
import seaborn as sns

# Count
from collections import Counter

In [ ]:
class CFG:
    SEED = 42

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/test.csv')
snp_info = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/snp_info.csv')

In [ ]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [ ]:
train_x, y_train = get_x_y(train)
test_x = get_x_y(test)

In [ ]:
X_train = train_x.iloc[:,3:]
X_test = test_x.iloc[:,3:]

## Feature_set_01

### 파생변수생성

#### SNP 분할

In [ ]:
np.unique(X_train.iloc[:,1:].values)

array(['A A', 'A G', 'C A', 'C C', 'G A', 'G G'], dtype=object)

In [ ]:
X_list = X_train.iloc[:,1:].columns.to_list()

In [ ]:
for i in X_list:
    X_train[f'{i}_0'] = X_train[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[0]
for i in X_list:
    X_train[f'{i}_1'] = X_train[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[1]

In [ ]:
for i in X_list:
    X_test[f'{i}_0'] = X_test[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[0]
for i in X_list:
    X_test[f'{i}_1'] = X_test[f'{i}'].astype('str').apply(lambda x: x.split(' ')).str[1]

In [ ]:
X_train.iloc[:,16:].head()

,SNP_01_0,SNP_02_0,SNP_03_0,SNP_04_0,SNP_05_0,SNP_06_0,SNP_07_0,SNP_08_0,SNP_09_0,SNP_10_0,...,SNP_06_1,SNP_07_1,SNP_08_1,SNP_09_1,SNP_10_1,SNP_11_1,SNP_12_1,SNP_13_1,SNP_14_1,SNP_15_1
0,G,A,A,G,C,A,A,G,A,G,...,A,A,G,A,G,G,A,A,A,A
1,A,A,C,A,A,A,A,G,A,A,...,G,A,A,A,G,A,A,G,A,A
2,G,G,A,G,C,G,A,G,G,A,...,G,A,A,A,G,A,A,A,A,A
3,A,G,A,G,A,G,G,A,G,A,...,G,G,A,G,G,G,G,G,A,G
4,G,G,C,A,C,A,A,A,A,G,...,A,A,A,A,G,A,A,G,A,A


#### SNP 합

In [ ]:
chrom = pd.DataFrame(snp_info.iloc[:,2])
cm = pd.DataFrame(snp_info.iloc[:,3])
pos = pd.DataFrame(snp_info.iloc[:,4])
chorm_cm = pd.concat([chrom,cm],axis=1)
chorm_pos = pd.concat([chrom,pos],axis=1)
cm_pos = pd.concat([chrom,cm],axis=1)
chrom_cm_pos = pd.concat([chrom,cm,pos],axis=1)
info_list = [chrom,cm,pos,chorm_cm,chorm_pos,cm_pos,chrom_cm_pos]

In [ ]:
predict = pd.DataFrame()
for i in range(len(info_list)):
  model = KMeans(n_clusters=6, random_state=CFG.SEED)
  model.fit(info_list[i])
  predict = pd.concat([predict,pd.DataFrame(model.predict(info_list[i]))],axis=1)
predict.columns = ['chrom_clust','cm_clust','pos_clust','chorm_cm_clust','chorm_pos_clust','cm_pos_clust','chrom_cm_pos_clust']
snp_info = pd.concat([snp_info,predict], axis=1)

In [ ]:
chorm_list = []
cm_list = []
pos_list = []
chrom_cm_list = []
chrom_pos_list = []
cm_pos_list = []
chrom_cm_pos_list = []

for i in range(6):
  chorm_list.append(snp_info.query(f'chrom_clust == {i}').SNP_id.to_list())
for i in range(6):
  cm_list.append(snp_info.query(f'cm_clust == {i}').SNP_id.to_list())
for i in range(6):
  pos_list.append(snp_info.query(f'pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_cm_list.append(snp_info.query(f'chorm_cm_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_pos_list.append(snp_info.query(f'chorm_pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  cm_pos_list.append(snp_info.query(f'cm_pos_clust == {i}').SNP_id.to_list())
for i in range(6):
  chrom_cm_pos_list.append(snp_info.query(f'chrom_cm_pos_clust == {i}').SNP_id.to_list())

In [ ]:
all_list = [chorm_list,cm_list,pos_list,chrom_cm_list,chrom_pos_list,cm_pos_list,chrom_cm_pos_list]
all_columns = predict.columns.to_list()

In [ ]:
for j in range(7):
  for i in range(6):
    if len(all_list[j][i]) == 8:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]] + ' ' + X_train[all_list[j][i][6]] + ' ' + X_train[all_list[j][i][7]]
    elif len(all_list[j][i]) == 7:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]] + ' ' + X_train[all_list[j][i][6]]
    elif len(all_list[j][i]) == 6:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]] + ' ' + X_train[all_list[j][i][5]]
    elif len(all_list[j][i]) == 5:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]] + ' ' + X_train[all_list[j][i][4]]
    elif len(all_list[j][i]) == 4:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]] + ' ' + X_train[all_list[j][i][3]]
    elif len(all_list[j][i]) == 3:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]] + ' ' + X_train[all_list[j][i][2]]
    elif len(all_list[j][i]) == 2:
      X_train[f'{all_columns[j]}{i}'] = X_train[all_list[j][i][0]] + ' ' + X_train[all_list[j][i][1]]

In [ ]:
for j in range(7):
  for i in range(6):
    if len(all_list[j][i]) == 8:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]] + ' ' + X_test[all_list[j][i][6]] + ' ' + X_test[all_list[j][i][7]]
    elif len(all_list[j][i]) == 7:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]] + ' ' + X_test[all_list[j][i][6]]
    elif len(all_list[j][i]) == 6:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]] + ' ' + X_test[all_list[j][i][5]]
    elif len(all_list[j][i]) == 5:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]] + ' ' + X_test[all_list[j][i][4]]
    elif len(all_list[j][i]) == 4:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]] + ' ' + X_test[all_list[j][i][3]]
    elif len(all_list[j][i]) == 3:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]] + ' ' + X_test[all_list[j][i][2]]
    elif len(all_list[j][i]) == 2:
      X_test[f'{all_columns[j]}{i}'] = X_test[all_list[j][i][0]] + ' ' + X_test[all_list[j][i][1]]

In [ ]:
X_train.iloc[:,46:].head()

,chrom_clust0,chrom_clust4,cm_clust0,cm_clust4,pos_clust0,pos_clust2,pos_clust4,chorm_cm_clust0,chorm_cm_clust3,chorm_cm_clust5,chorm_pos_clust0,chorm_pos_clust2,chorm_pos_clust4,cm_pos_clust0,cm_pos_clust3,cm_pos_clust5,chrom_cm_pos_clust0,chrom_cm_pos_clust1,chrom_cm_pos_clust3
0,A G A A G A C A A A A A G G A A,A A A A A A,G G A A G G A A A A A A,G A C A A A A A G G,G A C A A A A A A A A A,G G A A G G,G G A A A A,G G A A A A A A,G A C A A A A A G G A A,G G A A,G A C A A A A A A A A A,G G A A G G,G G A A A A,G G A A A A A A,G A C A A A A A G G A A,G G A A,G G A A G G,G G A A A A,G A C A A A A A A A A A
1,A G C A A A A A A G A A G A A A,G A G G A A,A G C A A G G A G G A A,A A A A A G A A G A,A A A A A G A A G A G G,A G C A A G,G A A A A A,A G G A G G A A,A A A A A G A A G A A A,A G C A,A A A A A G A A G A G G,A G C A A G,G A A A A A,A G G A G G A A,A A A A A G A A G A A A,A G C A,A G C A A G,G A A A A A,A A A A A G A A G A G G
2,G G A A G A C C G G A A G A G A,A A A A A A,G G A A A G A A A A A A,G A C C G G A A G A,G A C C G G A A A A A A,G G A A A G,G A G A A A,A G A A A A A A,G A C C G G A A G A G A,G G A A,G A C C G G A A A A A A,G G A A A G,G A G A A A,A G A A A A A A,G A C C G G A A G A G A,G G A A,G G A A A G,G A G A A A,G A C C G G A A A A A A
3,G G A A G A A A G G G G A A G G,G G G G A A,A A A A A G G G G G A A,G A A A G G G G A A,G A A A G G G G G G G G,A A A A A G,A A G G A A,A G G G G G A A,G A A A G G G G A A G G,A A A A,G A A A G G G G G G G G,A A A A A G,A A G G A A,A G G G G G A A,G A A A G G G G A A G G,A A A A,A A A A A G,A A G G A A,G A A A G G G G G G G G
4,G G C C A A C C A A A A A A A A,A A A G A A,G G C C G G A A A G A A,A A C C A A A A A A,A A C C A A A A A A A G,G G C C G G,A A A A A A,G G A A A G A A,A A C C A A A A A A A A,G G C C,A A C C A A A A A A A G,G G C C G G,A A A A A A,G G A A A G A A,A A C C A A A A A A A A,G G C C,G G C C G G,A A A A A A,A A C C A A A A A A A G


#### SNP A,C,G 개수

In [ ]:
X_train['A_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['A']
X_train['C_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['C']
X_train['G_count'] = Counter(X_train.iloc[:,16:46].iloc[0,:])['G']

In [ ]:
X_test['A_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['A']
X_test['C_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['C']
X_test['G_count'] = Counter(X_test.iloc[:,16:46].iloc[0,:])['G']

In [ ]:
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['A_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['A']
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['C_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['C']
for i in range(len(X_train.iloc[:,16:].index)):
    X_train['G_count'].iloc[i] = Counter(X_train.iloc[:,16:46].iloc[i,:])['G']

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['A_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['A']
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['C_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['C']
for i in range(len(X_test.iloc[:,16:].index)):
    X_test['G_count'].iloc[i] = Counter(X_test.iloc[:,16:46].iloc[i,:])['G']

In [ ]:
X_train['A+C_count'] = X_train['A_count']+X_train['C_count']
X_train['A+G_count'] = X_train['A_count']+X_train['G_count']
X_train['C+G_count'] = X_train['C_count']+X_train['G_count']

In [ ]:
X_test['A+C_count'] = X_test['A_count']+X_test['C_count']
X_test['A+G_count'] = X_test['A_count']+X_test['G_count']
X_test['C+G_count'] = X_test['C_count']+X_test['G_count']

In [ ]:
X_train.iloc[:,65:].head()

,A_count,C_count,G_count,A+C_count,A+G_count,C+G_count
0,20,1,9,21,29,10
1,21,1,8,22,29,9
2,18,2,10,20,28,12
3,12,0,18,12,30,18
4,18,4,8,22,26,12


#### trait 문자형 변환

In [ ]:
X_train = X_train.astype(str)
X_test = X_test.astype(str)

In [ ]:
X_train.shape, X_test.shape, y_train.shape

((262, 71), (175, 71), (262,))

In [ ]:
X_train_01 = X_train.copy()
X_test_01 = X_test.copy()

## Feature_set_02

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/test.csv')
snp = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/snp_info.csv')

In [ ]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [ ]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [ ]:
train_x.drop(columns=['father','mother','gender'],inplace=True)
test_x.drop(columns=['father','mother','gender'],inplace=True)

In [ ]:
train_x['2_BTA'] = train_x['SNP_01']
train_x['chrom_6'] = train_x['SNP_02'] + '-'+ train_x['SNP_03'] + '-' + train_x['SNP_04'] + '-' + train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_07'] + '-' + train_x['SNP_08'] + '-' + train_x['SNP_09']
train_x['6_ARS_Parent'] = train_x['SNP_02']
train_x['6_ARS_BFGL'] = train_x['SNP_03'] + '-' + train_x['SNP_04'] + '-' + train_x['SNP_09']
train_x['6_BOVINE'] = train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_08']
train_x['6_HAPMAP'] = train_x['SNP_07']
train_x['7_BTB'] = train_x['SNP_10']
train_x['8_ARS'] = train_x['SNP_11']
train_x['chrom_9'] = train_x['SNP_12'] + '-' + train_x['SNP_13'] + '-' + train_x['SNP_14']
train_x['9_HAPMAP'] = train_x['SNP_12'] + '-' + train_x['SNP_14']
train_x['9_BTB'] = train_x['SNP_13']
train_x['10_BOVINE'] = train_x['SNP_15']
train_x['SNP_total'] = train_x['SNP_01'] + '-' + train_x['SNP_02'] + '-' + train_x['SNP_03'] + '-' + train_x['SNP_04'] + '-' + train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_07'] + '-' + train_x['SNP_08'] + '-' + train_x['SNP_09'] + '-' + train_x['SNP_10'] + '-' + train_x['SNP_11'] + '-' + train_x['SNP_12'] + '-' + train_x['SNP_13'] + '-' + train_x['SNP_14'] + '-' + train_x['SNP_15']  

In [ ]:
test_x['2_BTA'] = test_x['SNP_01']
test_x['chrom_6'] = test_x['SNP_02'] + '-'+ test_x['SNP_03'] + '-' + test_x['SNP_04'] + '-' + test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_07'] + '-' + test_x['SNP_08'] + '-' + test_x['SNP_09']
test_x['6_ARS_Parent'] = test_x['SNP_02']
test_x['6_ARS_BFGL'] = test_x['SNP_03'] + '-' + test_x['SNP_04'] + '-' + test_x['SNP_09']
test_x['6_BOVINE'] = test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_08']
test_x['6_HAPMAP'] = test_x['SNP_07']
test_x['7_BTB'] = test_x['SNP_10']
test_x['8_ARS'] = test_x['SNP_11']
test_x['chrom_9'] = test_x['SNP_12'] + '-' + test_x['SNP_13'] + '-' + test_x['SNP_14']
test_x['9_HAPMAP'] = test_x['SNP_12'] + '-' + test_x['SNP_14']
test_x['9_BTB'] = test_x['SNP_13']
test_x['10_BOVINE'] = test_x['SNP_15']
test_x['SNP_total'] = test_x['SNP_01'] + '-' + test_x['SNP_02'] + '-' + test_x['SNP_03'] + '-' + test_x['SNP_04'] + '-' + test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_07'] + '-' + test_x['SNP_08'] + '-' + test_x['SNP_09'] + '-' + test_x['SNP_10'] + '-' + test_x['SNP_11'] + '-' + test_x['SNP_12'] + '-' + test_x['SNP_13'] + '-' + test_x['SNP_14'] + '-' + test_x['SNP_15']  

In [ ]:
train_x['2_BTA'] = train_x['2_BTA'].apply(lambda x : x.replace(" ",""))
train_x['chrom_6'] = train_x['chrom_6'].apply(lambda x : x.replace(" ",""))
train_x['6_ARS_Parent'] = train_x['6_ARS_Parent'].apply(lambda x : x.replace(" ",""))
train_x['6_ARS_BFGL'] = train_x['6_ARS_BFGL'].apply(lambda x : x.replace(" ",""))
train_x['6_BOVINE'] = train_x['6_BOVINE'].apply(lambda x : x.replace(" ",""))
train_x['6_HAPMAP'] = train_x['6_HAPMAP'].apply(lambda x : x.replace(" ",""))
train_x['7_BTB'] = train_x['7_BTB'].apply(lambda x : x.replace(" ",""))
train_x['8_ARS'] = train_x['8_ARS'].apply(lambda x : x.replace(" ",""))
train_x['chrom_9'] = train_x['chrom_9'].apply(lambda x : x.replace(" ",""))
train_x['9_HAPMAP'] = train_x['9_HAPMAP'].apply(lambda x : x.replace(" ",""))
train_x['9_BTB'] = train_x['9_BTB'].apply(lambda x : x.replace(" ",""))
train_x['10_BOVINE'] = train_x['10_BOVINE'].apply(lambda x : x.replace(" ",""))
train_x['SNP_total'] = train_x['SNP_total'].apply(lambda x : x.replace(" ",""))

In [ ]:
test_x['2_BTA'] = test_x['2_BTA'].apply(lambda x : x.replace(" ",""))
test_x['chrom_6'] = test_x['chrom_6'].apply(lambda x : x.replace(" ",""))
test_x['6_ARS_Parent'] = test_x['6_ARS_Parent'].apply(lambda x : x.replace(" ",""))
test_x['6_ARS_BFGL'] = test_x['6_ARS_BFGL'].apply(lambda x : x.replace(" ",""))
test_x['6_BOVINE'] = test_x['6_BOVINE'].apply(lambda x : x.replace(" ",""))
test_x['6_HAPMAP'] = test_x['6_HAPMAP'].apply(lambda x : x.replace(" ",""))
test_x['7_BTB'] = test_x['7_BTB'].apply(lambda x : x.replace(" ",""))
test_x['8_ARS'] = test_x['8_ARS'].apply(lambda x : x.replace(" ",""))
test_x['chrom_9'] = test_x['chrom_9'].apply(lambda x : x.replace(" ",""))
test_x['9_HAPMAP'] = test_x['9_HAPMAP'].apply(lambda x : x.replace(" ",""))
test_x['9_BTB'] = test_x['9_BTB'].apply(lambda x : x.replace(" ",""))
test_x['10_BOVINE'] = test_x['10_BOVINE'].apply(lambda x : x.replace(" ",""))
test_x['SNP_total'] = test_x['SNP_total'].apply(lambda x : x.replace(" ",""))

In [ ]:
train_x.columns = list(pd.RangeIndex(len(train_x.columns)))
test_x.columns = list(pd.RangeIndex(len(test_x.columns)))

In [ ]:
data =[]
for i in range(0,len(train_x)):
        p= train_x[1][i]+train_x[2][i]+train_x[3][i]+train_x[4][i]+train_x[5][i]+train_x[6][i]+train_x[7][i]+train_x[8][i]+train_x[9][i]+train_x[10][i]+train_x[11][i]+train_x[12][i]+train_x[13][i]+train_x[14][i]+train_x[15][i]
        p = p.replace(" ","")
        data.append(p)
        
train_x['concat'] = data

numGC =[0]*len(train_x)
numA =[0]*len(train_x)
sub =[0]*len(train_x)
H = [0]*len(train_x)

for i in range(0,len(train_x)):
    for j in range(0,30):
        if train_x['concat'][i][j] != 'A':
            numGC[i] = numGC[i]+1
        else:
            numA[i] = numA[i]+1
train_x['numGC'] = numGC
train_x['numA'] = numA

for i in range(0,len(train_x)):
    sub[i] = numGC[i]-numA[i]
    
for i in range(0,len(train_x)):
    H[i] = numGC[i]*3+numA[i]*2

train_x['sub'] = sub
train_x['H'] = H

In [ ]:
data =[]
for i in range(0,len(test_x)):
        p= test_x[1][i]+test_x[2][i]+test_x[3][i]+test_x[4][i]+test_x[5][i]+test_x[6][i]+test_x[7][i]+test_x[8][i]+test_x[9][i]+test_x[10][i]+test_x[11][i]+test_x[12][i]+test_x[13][i]+test_x[14][i]+test_x[15][i]
        p = p.replace(" ","")
        data.append(p)
        
test_x['concat'] = data

numGC =[0]*len(test_x)
numA =[0]*len(test_x)
sub =[0]*len(test_x)
H = [0]*len(test_x)

for i in range(0,len(test_x)):
    for j in range(0,30):
        if test_x['concat'][i][j] != 'A':
            numGC[i] = numGC[i]+1
        else:
            numA[i] = numA[i]+1
test_x['numGC'] = numGC
test_x['numA'] = numA

for i in range(0,len(test_x)):
    sub[i] = numGC[i]-numA[i]
    
for i in range(0,len(test_x)):
    H[i] = numGC[i]*3+numA[i]*2

test_x['sub'] = sub
test_x['H'] = H

In [ ]:
train_x = train_x.astype(str)
test_x = test_x.astype(str)

In [ ]:
X_train_02 = train_x.copy()
X_test_02 = test_x.copy()

## Feature_set_03

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/test.csv')
snp = pd.read_csv('/content/drive/MyDrive/유전체공모전/data/snp_info.csv')

In [ ]:
def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

In [ ]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [ ]:
train_x.drop(columns=['father','mother','gender'],inplace=True)
test_x.drop(columns=['father','mother','gender'],inplace=True)

In [ ]:
#SNP combine(chrom)
train_x['chrom_2'] = train_x['SNP_01']
train_x['chrom_6'] = train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_07'] + '-' + train_x['SNP_08'] + '-' + train_x['SNP_09']
train_x['chrom_7'] = train_x['SNP_10']
train_x['chrom_8'] = train_x['SNP_11']
train_x['chrom_9'] = train_x['SNP_12'] + '-' + train_x['SNP_13'] + '-' + train_x['SNP_14']
train_x['chrom_10'] = train_x['SNP_15']
train_x['SNP_total'] = train_x['SNP_01'] + '-' + train_x['SNP_02'] + '-' + train_x['SNP_03'] + '-' + train_x['SNP_04'] + '-' + train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_07'] + '-' + train_x['SNP_08'] + '-' + train_x['SNP_09'] + '-' + train_x['SNP_10'] + '-' + train_x['SNP_11'] + '-' + train_x['SNP_12'] + '-' + train_x['SNP_13'] + '-' + train_x['SNP_14'] + '-' + train_x['SNP_15']  

test_x['chrom_2'] = test_x['SNP_01']
test_x['chrom_6'] = test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_07'] + '-' + test_x['SNP_08'] + '-' + test_x['SNP_09']
test_x['chrom_7'] = test_x['SNP_10']
test_x['chrom_8'] = test_x['SNP_11']
test_x['chrom_9'] = test_x['SNP_12'] + '-' + test_x['SNP_13'] + '-' + test_x['SNP_14']
test_x['chrom_10'] = test_x['SNP_15']
test_x['SNP_total'] = test_x['SNP_01'] + '-' + test_x['SNP_02'] + '-' + test_x['SNP_03'] + '-' + test_x['SNP_04'] + '-' + test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_07'] + '-' + test_x['SNP_08'] + '-' + test_x['SNP_09'] + '-' + test_x['SNP_10'] + '-' + test_x['SNP_11'] + '-' + test_x['SNP_12'] + '-' + test_x['SNP_13'] + '-' + test_x['SNP_14'] + '-' + test_x['SNP_15']

In [ ]:
#SNP combine(cm)
train_x['cm_60'] = train_x['SNP_01'] + '-' + train_x['SNP_03'] + '-' + train_x['SNP_10'] + '-' + train_x['SNP_12'] + '-' + train_x['SNP_13'] + '-' + train_x['SNP_14']

test_x['cm_60'] = test_x['SNP_01'] + '-' + test_x['SNP_03'] + '-' + test_x['SNP_10'] + '-' + test_x['SNP_12'] + '-' + test_x['SNP_13'] + '-' + test_x['SNP_14']

In [ ]:
#SNP combine(pos)
train_x['pos_4'] = train_x['SNP_01'] + '-' + train_x['SNP_03'] + '-' + train_x['SNP_10']
train_x['pos_5'] = train_x['SNP_04'] + '-' + train_x['SNP_12'] + '-' + train_x['SNP_13']
train_x['pos_6'] = train_x['SNP_05'] + '-' + train_x['SNP_06'] + '-' + train_x['SNP_07'] + '-' + train_x['SNP_08']
train_x['pos_7'] = train_x['SNP_09'] + '-' + train_x['SNP_14']

test_x['pos_4'] = test_x['SNP_01'] + '-' + test_x['SNP_03'] + '-' + test_x['SNP_10']
test_x['pos_5'] = test_x['SNP_04'] + '-' + test_x['SNP_12'] + '-' + test_x['SNP_13']
test_x['pos_6'] = test_x['SNP_05'] + '-' + test_x['SNP_06'] + '-' + test_x['SNP_07'] + '-' + test_x['SNP_08']
test_x['pos_7'] = test_x['SNP_09'] + '-' + test_x['SNP_14']

In [ ]:
train_x['chrom_6'] = train_x['chrom_6'].apply(lambda x : x.replace(" ",""))
train_x['chrom_9'] = train_x['chrom_9'].apply(lambda x : x.replace(" ",""))
train_x['SNP_total'] = train_x['SNP_total'].apply(lambda x : x.replace(" ",""))

train_x['cm_60'] = train_x['cm_60'].apply(lambda x : x.replace(" ",""))

train_x['pos_4'] = train_x['pos_4'].apply(lambda x : x.replace(" ",""))
train_x['pos_5'] = train_x['pos_5'].apply(lambda x : x.replace(" ",""))
train_x['pos_6'] = train_x['pos_6'].apply(lambda x : x.replace(" ",""))
train_x['pos_7'] = train_x['pos_7'].apply(lambda x : x.replace(" ",""))

test_x['chrom_6'] = test_x['chrom_6'].apply(lambda x : x.replace(" ",""))
test_x['chrom_9'] = test_x['chrom_9'].apply(lambda x : x.replace(" ",""))
test_x['SNP_total'] = test_x['SNP_total'].apply(lambda x : x.replace(" ",""))

test_x['cm_60'] = test_x['cm_60'].apply(lambda x : x.replace(" ",""))

test_x['pos_4'] = test_x['pos_4'].apply(lambda x : x.replace(" ",""))
test_x['pos_5'] = test_x['pos_5'].apply(lambda x : x.replace(" ",""))
test_x['pos_6'] = test_x['pos_6'].apply(lambda x : x.replace(" ",""))
test_x['pos_7'] = test_x['pos_7'].apply(lambda x : x.replace(" ",""))

In [ ]:
train_x.drop(columns=['SNP_01','SNP_10','SNP_11','SNP_15'],inplace=True)
test_x.drop(columns=['SNP_01','SNP_10','SNP_11','SNP_15'],inplace=True)

In [ ]:
train_x = train_x.astype(str)
test_x = test_x.astype(str)

In [ ]:
X_train_03 = train_x.copy()
X_test_03 = test_x.copy()

In [ ]:
X_train = pd.concat([X_train_01,X_train_02,X_train_03],axis=1)
X_test = pd.concat([X_test_01,X_test_02,X_test_03],axis=1)

In [ ]:
X_train.columns = list(pd.RangeIndex(len(X_train.columns)))
X_train = X_train.loc[:,~X_train.T.duplicated()]
X_train.columns = list(pd.RangeIndex(len(X_train.columns)))
X_test.columns = list(pd.RangeIndex(len(X_test.columns)))
X_test = X_test.loc[:,~X_test.T.duplicated()]
X_test.columns = list(pd.RangeIndex(len(X_test.columns)))

In [ ]:
X_train.shape, X_test.shape, y_train.shape

((262, 84), (175, 84), (262,))

## Submission

In [ ]:
# X_train.to_csv('/content/drive/MyDrive/유전체공모전/data/X_train_03.csv', index=False)
# X_test.to_csv('/content/drive/MyDrive/유전체공모전/data/X_test_03.csv', index=False)
# y_train.to_csv('/content/drive/MyDrive/유전체공모전/data/y_train_03.csv', index=False)